In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator

Parameters to adjust
UMAP - n_neighbors, min_dist (0 - 0.1), metric (cosine)
HDBSCAN - min_cluster_size, min_samples

Metrics to consider
Accuracy

TRACK The Results

In [3]:
# DEPRECATED
# chroma_client = chromadb.PersistentClient(path="./../../Clustering/Clustering/Chroma")
# # Count number of collections
# print(chroma_client.count_collections())
# 
# # Get all collection names
# collection_names = chroma_client.list_collections()
# 
# # Loop through each collection and drop it
# for collection_name in collection_names:
#     if collection_name.name != 'climate_claims_embeddings_unchanged':
#         chroma_client.delete_collection(collection_name.name)

In [14]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
for param in params:
    percentage = 1
    data_loader = DataLoader(percentage, True, param['random_seed'])
    train_df, test_df = data_loader.create_train_test_df(True, True, True)
    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    print(best_estimator.get_all_performance_metrics())
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
    }
    print(output)
    results.append(output)

Numerical Rating
1    2990
3     619
Name: count, dtype: int64
Numerical Rating
1    778
3    124
Name: count, dtype: int64
The_past_4_years_have_seen_12_major_landfalling_hurricanes,_very_low_but_not_unprecedented_?.
Had to create embedding...
A_survey_by_the_Chicago_Council_on_Foreign_Relations_found_that_87%_of_the_public_favors_"working_through_the_United_Nations_to_strengthen_international_laws_against_terrorism.".
Had to create embedding...
YES_,_according_to_data_published_by_1015_individual_scientists_from_584_research_institutions_in_44_different_countries_and_counting!.
Had to create embedding...
This_comes_almost_exactly_40_years_after_the_government_wrote_a_state_of_the_climate_report_saying_that_global_cooling_was_going_to_cause_floods,_famines,_extreme_weather,_and_wouldkill_us_all.._First,_74%_of_all_U.S._stations_are_adjusted,_while_only_37%_of_ROW_stations_are_adjusted.
Had to create embedding...
The_result_was_starvation_for_many_of_the_bison.._A_new_paper_published_i

In [15]:
results_df = pd.json_normalize(results)
results_df

,percentage,score,accuracy,was_supervised_umap_used,metrics.accuracy,metrics.accuracy_not_including_fours,metrics.percentage_of_fours,metrics.percentage_of_no_clusters_in_ground_truth,metrics.precision_on_three,metrics.recall_on_three,...,metrics.recall_on_three_excluding_fours,metrics.accuracy_90_confidence,metrics.accuracy_80_confidence,metrics.accuracy_70_confidence,metrics.accuracy_60_confidence,metrics.percentage_90_confidence,metrics.percentage_80_confidence,metrics.percentage_70_confidence,metrics.percentage_60_confidence,metrics.cluster_df
0,1,1.673984,0.490022,True,0.490022,0.971429,0.495565,0.0,0.966667,0.233871,...,0.707317,0.4102,0.4102,0.4102,0.4102,0.965632,0.965632,0.965632,0.965632,...
